<H1><b>Unlimited Register Machine</b></H1>

# Machine

## Structure

In [1]:
control = None  # refers to a URM-program
memory  = None  # refers to the used memory
arity   = None  # refers to the input size

## Instructions

In [2]:
def Z(operand, line):
    # Z instruction
    global memory
    memory[operand] = 0
    return line + 1


def S(operand, line):
    # S instruction
    global memory
    memory[operand] += 1
    return line + 1


def C(operand1, operand2, line):
    # C instruction
    global memory
    memory[operand2] = memory[operand1]
    return line + 1


def J(operand1, operand2, operand3, line):
    # J instruction
    global memory
    if memory[operand1] == memory[operand2]:
        return operand3
    else:
        return line + 1

## Program Compiling

`Z(n)`       is represented by `(0, n)`<br/>
`S(n)`       is represented by `(1, n)`<br/>
`C(n, m)`    is represented by `(2, n, m)`<br/>
`J(n, m, q)` is represented by `(3, n, m, q)`

In [3]:
def compile(text):
    ok = isinstance(text, str)
    if not ok:
        raise ValueError("invalid program")
    def parse_line(line):
        """returns the instruction representation if 'line' is correct
        instruction and None otherwise"""
        cmd, _, line = line.partition('(')
        if cmd == 'Z' or cmd == 'S':
            operand, _, line = line.partition(')')
            try:
                operand = int(operand)
            except ValueError:
                return None
            return ((0 if cmd == 'Z' else 1, operand) if
                    operand >= 0 else None)
        elif cmd == 'C':
            try:
                operand, _, line = line.partition(',')
                operand1 = int(operand)
                operand, _, line = line.partition(')')
                operand2 = int(operand)
            except ValueError:
                return None
            return ((2, operand1, operand2) if
                    operand1 >= 0 and operand2 >= 0 else
                    None)
        elif cmd == 'J':
            try:
                operand, _, line = line.partition(',')
                operand1 = int(operand)
                operand, _, line = line.partition(',')
                operand2 = int(operand)
                operand, _, line = line.partition(')')
                operand3 = int(operand)
            except ValueError:
                return None
            return ((3, operand1, operand2, operand3) if
                    operand1 >= 0 and operand2 >= 0 and operand3 >= 0 else
                    None)
        else:  # unrecognized instruction
            return None
    # main function
    #
    # split text into list of lines
    lines = text.split('\n')
    # remove heading and tailing whitespaces in each line
    lines = [line.strip() for line in lines]
    # remove empty lines
    lines = [line for line in lines if line]
    # parsing each line
    program = [parse_line(line) for line in lines]
    # search error in the program
    errors = [None for item in program if item is None]
    if errors:  # program has errors
        raise ValueError("invalid program")
    return program

## Program Loading

In [4]:
def load(code, input_size):
    global memory, control, arity
    ok = isinstance(code, list)
    if not ok:
        raise ValueError("invalid code")
    ok = all(map(isinstance, code, len(code) * [tuple]))
    if not ok:
        raise ValueError("invalid code")
    ok = isinstance(input_size, int) and input_size >= 1
    if not ok:
        raise ValueError("invalid input size")
    high_memory_address = input_size
    for item in code:
        if (len(item) == 2 and
            (item[0] == 0 or item[0] == 1) and
            isinstance(item[1], int) and item[1] >= 0):
            high_memory_address = max(high_memory_address, item[1])
        elif (len(item) == 3 and item[0] == 2 and
              isinstance(item[1], int) and item[1] >= 0 and
              isinstance(item[2], int) and item[2] >= 0):
            high_memory_address = max(high_memory_address, item[1], item[2])
        elif (len(item) == 4 and item[0] == 3 and
              isinstance(item[1], int) and item[1] >= 0 and
              isinstance(item[2], int) and item[2] >= 0 and
              isinstance(item[3], int) and item[3] >= 0):
            high_memory_address = max(high_memory_address, item[1], item[2])
        else:
            raise ValueError("invalid code")
    memory = dict([(n, 0) for n in range(high_memory_address + 1)])
    control = code
    arity = input_size


## Program Performing

In [5]:
def run(*args):
    global memory, control, arity
    if control is None:
        raise ValueError("the program for running is not loaded")
    ok = len(args) == arity
    if not ok:
        raise ValueError("invalid number of arguments")
    ok = all(map(lambda x: isinstance(x, int) and x >= 0, args))
    if not ok:
        raise ValueError("invalid arguments")
    for k in memory:
        memory[k] = args[k - 1] if 1 <= k <= arity else 0
    ic = 1
    while ic >= 1:
        try:
            item = control[ic - 1]
            if item[0] == 0:
                ic = Z(item[1], ic)
            elif item[0] == 1:
                ic = S(item[1], ic)
            elif item[0] == 2:
                ic = C(item[1], item[2], ic)
            else:  # item[0] == 3
                ic = J(item[1], item[2], item[3], ic)
        except IndexError:
            break
    return memory[0]

In [6]:
# @title Computing the sum of n and m
n = 5 # @param {type:"integer"}
m = 25 # @param {type:"integer"}

if n < 0:
    print("Error: n is negative")
elif m < 0:
    print("Error: m is negative")
else:
    code = compile("""
    C(2,0)
    Z(2)
    J(1,2,0)
        S(0)
        S(2)
    J(0,0,3)
    """)
    load(code, 2)
    print(f"{n} + {m} = {run(n, m)}")

5 + 25 = 30


In [7]:
# @title Computing Fibonacci Series
length_of_series = 20 # @param {type:"integer"}

if length_of_series < 0:
    print("Error: length of series should be nonnegative")
elif length_of_series == 0:
    print("Nothing")
else:
    code = compile("""
    J(1,3,0)
    S(0)
    S(3)
    J(1,3,0)
        C(0,4)
        J(5,2,10)
        S(0)
        S(5)
    J(0,0,6)
    Z(5)
    C(4,2)
    S(3)
    J(0,0,4)
    """)
load(code,1)
for k in range(length_of_series):
    print(f"{'fibb(' + str(k) + ')':8} = {run(k)}")

fibb(0)  = 0
fibb(1)  = 1
fibb(2)  = 1
fibb(3)  = 2
fibb(4)  = 3
fibb(5)  = 5
fibb(6)  = 8
fibb(7)  = 13
fibb(8)  = 21
fibb(9)  = 34
fibb(10) = 55
fibb(11) = 89
fibb(12) = 144
fibb(13) = 233
fibb(14) = 377
fibb(15) = 610
fibb(16) = 987
fibb(17) = 1597
fibb(18) = 2584
fibb(19) = 4181


In [8]:
# @title Computing Function pred(n) = n - 1 if n > 0 else 0
number_of_attempt = 10 # @param {type:"integer"}

if number_of_attempt < 0:
    print("Error: n should be positive")
elif number_of_attempt == 0:
    print("Nothing")
else:
    code = compile("""
    J(0,1,0)
    S(2)
    J(1,2,0)
        S(0)
        S(2)
    J(0,0,3)
    """)
    load(code, 1)
    for n in range(number_of_attempt):
        print(f"x = {n:2} and pred(x) = {run(n):2}")

x =  0 and pred(x) =  0
x =  1 and pred(x) =  0
x =  2 and pred(x) =  1
x =  3 and pred(x) =  2
x =  4 and pred(x) =  3
x =  5 and pred(x) =  4
x =  6 and pred(x) =  5
x =  7 and pred(x) =  6
x =  8 and pred(x) =  7
x =  9 and pred(x) =  8


In [23]:
# @title Computing Function sub(n, m) = n - m if n > m else 0
n = 30 # @param {type:"integer"}
m = 25 # @param {type:"integer"}

if n < 0:
    print("Error: n should be nonnegative")
elif m < 0:
    print("Error: m should be nonnegative")
else:
    code = compile("""
    C(1,4)
    J(3,2,14)
        J(5,4,10)
        S(5)
        J(5,4,9)
            S(5)
            S(0)
        J(0,0,5)
        Z(5)
        S(3)
        C(0,4)
        Z(0)
    J(0,0,2)
    C(4,0)
    """)
    load(code, 2)
    print(f"sub({n}, {m}) = {run(n, m)}")

sub(30, 25) = 5


# Programming Technique

## Characteristics of a Program

In [10]:
def is_instruction(x):
    ok = isinstance(x, tuple)
    if not ok:
        return False
    if len(x) == 2:
        return (x[0] == 0 or x[0] == 1) and isinstance(x[1], int) and x[1] >= 0
    if len(x) == 3:
        return (x[0] == 2 and isinstance(x[1], int) and x[1] >= 0 and
                isinstance(x[2], int) and x[2] >= 0)
    if len(x) == 4:
        return (x[0] == 3 and isinstance(x[1], int) and x[1] >= 0 and
                isinstance(x[2], int) and x[2] >= 0 and
                isinstance(x[3], int) and x[3] >= 0)
    return False

def is_program(x):
    return isinstance(x, list) and all(map(is_instruction,x))

In [11]:
def size(x):
    ok = is_program(x)
    if not ok:
        raise ValueError("invalid program")
    return len(x)

def haddr(x):
    ok = is_program(x)
    if not ok:
        raise ValueError("invalid program")
    temp = -1
    for i in x:
        if len(i) == 2:
            temp = max(temp, i[1])
        else:
            temp = max(temp, i[1], i[2])
    return temp if temp >= 0 else None

def normalize(P):
    ok = is_program(P)
    if not ok:
        raise ValueError("invalid argument")
    if not P:
        return P
    last_instr = size(P)
    normalized_P = []
    for i in P:
        if 0 <= i[0] <= 2 or 0 < i[3] <= last_instr:
            normalized_P.append(i)
        else:
            normalized_P.append((i[0], i[1], i[2], last_instr + 1))
    return normalized_P

## Concatenation of Programs

In [12]:
def pipe(P, Q):
    ok = is_program(P) and is_program(Q)
    if not ok:
        raise ValueError("invalid argument")
    if not P:
        return Q
    temp = normalize(P)
    off = size(P)
    for i in Q:
        if 0 <= i[0] <= 2 or i[3] == 0:
            temp.append(i)
        else:
            temp.append((i[0], i[1], i[2], i[3] + off))
    return temp

In [13]:
text1 = "Z(0)\nS(1)\nC(2,3)\nJ(4,5,0)"
text2 = "J(4,5,0)\nC(2,3)\nS(1)\nZ(0)"
code1, code2 = compile(text1), compile(text2)
code = pipe(code1, code2)
for line in range(size(code1)):
    print(f"first: {str(code1[line]):13}\tsecond: {13 * ' '}\t"
          f"concatenation: {code[line]}")
for line in range(size(code2)):
    print(f"first: {13 * ' '}\tsecond: {str(code2[line]):13}\t"
          f"concatenation: {code[size(code1) + line]}")


first: (0, 0)       	second:              	concatenation: (0, 0)
first: (1, 1)       	second:              	concatenation: (1, 1)
first: (2, 2, 3)    	second:              	concatenation: (2, 2, 3)
first: (3, 4, 5, 0) 	second:              	concatenation: (3, 4, 5, 5)
first:              	second: (3, 4, 5, 0) 	concatenation: (3, 4, 5, 0)
first:              	second: (2, 2, 3)    	concatenation: (2, 2, 3)
first:              	second: (1, 1)       	concatenation: (1, 1)
first:              	second: (0, 0)       	concatenation: (0, 0)


## Relocation of Program

In [14]:
def reloc(P, alloc):
    ok = is_program(P)
    if not ok:
        raise ValueError("invalid program")
    ok = isinstance(alloc, tuple) and len(alloc) == haddr(P) + 1
    if not ok:
        raise ValueError("invalid allocation")
    temp = []
    for i in P:
        if i[0] == 0 or i[0] == 1:
            temp.append((i[0], alloc[i[1]]))
            continue
        if i[0] == 2:
            temp.append((2, alloc[i[1]], alloc[i[2]]))
            continue
        # i[0] == 3
        temp.append((3, alloc[i[1]], alloc[i[2]], i[3]))
    return temp

In [15]:
new_code = reloc(code, (6, 7, 8, 9, 10, 11))
for line in range(size(code)):
    print(f"{'instruction before relocation: ' + str(code[line]):44}\t"
          f"{'instruction after relocation: ' + str(new_code[line]):43}")

instruction before relocation: (0, 0)       	instruction after relocation: (0, 6)       
instruction before relocation: (1, 1)       	instruction after relocation: (1, 7)       
instruction before relocation: (2, 2, 3)    	instruction after relocation: (2, 8, 9)    
instruction before relocation: (3, 4, 5, 5) 	instruction after relocation: (3, 10, 11, 5)
instruction before relocation: (3, 4, 5, 0) 	instruction after relocation: (3, 10, 11, 0)
instruction before relocation: (2, 2, 3)    	instruction after relocation: (2, 8, 9)    
instruction before relocation: (1, 1)       	instruction after relocation: (1, 7)       
instruction before relocation: (0, 0)       	instruction after relocation: (0, 6)       
